In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
cd gdrive

[Errno 2] No such file or directory: 'gdrive'
/content/gdrive/MyDrive/Colab datasets


In [ ]:
cd MyDrive

/content/gdrive/MyDrive


In [ ]:
cd Colab datasets

/content/gdrive/MyDrive/Colab datasets


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as mplot

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

import math

In [ ]:
Data = pd.read_csv("processed.csv")
Data.head()

,Category,CounterSpeech,commentText,date,hasReplies,id,likes,numberOfReplies,timestamp,user,Community,replies
0,Default,False,last brave white women brain,5 years ago,0.0,Ugw-_mfnTo9SUniGGTN4AaABAg,0,0.0,2012-12-05 09:13:58.535,Ik hou van surfen,jews,[]
1,Default,False,amen pastor,2 years ago,0.0,UgiMOgO3u4akkngCoAEC,0,0.0,2015-12-05 04:47:36.420,julius smith,lgbt,[]
2,"2,8",True,kind god hates people curses kind people belie...,4 years ago,0.0,UgzemCMbg-ri1ri7sCt4AaABAg,0,0.0,2013-12-05 09:16:05.293,gary wersley,jews,[]
3,8,True,question comment racist jews smartest group pe...,3 years ago,0.0,UghFhzrURF1svHgCoAEC,0,0.0,2015-01-05 16:27:25.589,Les Ordway,jews,[]
4,8,True,israel fail nothing new israel rebelious natio...,5 years ago,0.0,Ugzr50sQdH0Ictuex214AaABAg,0,0.0,2012-12-05 09:17:14.596,HamalachMichael,jews,[]


In [ ]:
Data.dropna(axis=0, inplace=True)

In [ ]:
Data = pd.read_csv("processed_correct_spelled.csv")
Data.head()

,Category,CounterSpeech,commentText,date,hasReplies,id,likes,numberOfReplies,timestamp,user,Community,replies
0,Default,False,last brave white women brain,5 years ago,0.0,Ugw-_mfnTo9SUniGGTN4AaABAg,0,0.0,2012-12-05 09:13:58.535,Ik hou van surfen,jews,[]
1,Default,False,amen pastor,2 years ago,0.0,UgiMOgO3u4akkngCoAEC,0,0.0,2015-12-05 04:47:36.420,julius smith,lgbt,[]
2,"2,8",True,kind god hates people curses kind people belie...,4 years ago,0.0,UgzemCMbg-ri1ri7sCt4AaABAg,0,0.0,2013-12-05 09:16:05.293,gary wersley,jews,[]
3,8,True,question comment racist jews smartest group pe...,3 years ago,0.0,UghFhzrURF1svHgCoAEC,0,0.0,2015-01-05 16:27:25.589,Les Ordway,jews,[]
4,8,True,israel fail nothing new israel rebellious nati...,5 years ago,0.0,Ugzr50sQdH0Ictuex214AaABAg,0,0.0,2012-12-05 09:17:14.596,HamalachMichael,jews,[]


In [ ]:
Data.dropna(axis=0, inplace=True)

In [ ]:
!pip install pyspellchecker

#spelling checker
from spellchecker import SpellChecker

# Instantiate spell checker
spell = SpellChecker()

# Correcting the spelling
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)

# Spellcheck data sets
Data.commentText = Data.commentText.apply(correct_spellings)

     |████████████████████████████████| 2.7MB 4.9MB/s 


In [ ]:
import random
from random import shuffle
random.seed(1)
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet 

def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list =words
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			#print("replaced", random_word, "with", synonym)
			num_replaced += 1
		if num_replaced >= n: #only replace up to n words
			break

	sentence = ' '.join(new_words)
	new_words = sentence.split(' ')

	return new_words

def get_synonyms(word):
	synonyms = set()
	for syn in wordnet.synsets(word): 
		for l in syn.lemmas(): 
			synonym = l.name().replace("_", " ").replace("-", " ").lower()
			synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
			synonyms.add(synonym) 
	if word in synonyms:
		synonyms.remove(word)
	return list(synonyms)

#====================================Random deletion==================================

def random_deletion(words, p):

	#obviously, if there's only one word, don't delete it
	if len(words) == 1:
		return words

	#randomly delete words with probability p
	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)

	#if you end up deleting all words, just return a random word
	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]

	return new_words

#========================================Random swap========================================

def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)
	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0
	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words
	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1] 
	return new_words

#==================================Random insertion==========================================

def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)
	return new_words

def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		random_word = new_words[random.randint(0, len(new_words)-1)]
		synonyms = get_synonyms(random_word)
		counter += 1
		if counter >= 10:
			return
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)

#==============================main data augmentation function======================================

def eda(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=4):
	
	words = sentence.split(' ')
	num_words = len(words)
	
	augmented_sentences = []
	num_new_per_technique = int(num_aug/4)+1

	#sr
	if (alpha_sr > 0):
		n_sr = max(1, int(alpha_sr*num_words))
		for _ in range(num_new_per_technique):
			a_words = synonym_replacement(words, n_sr)
			augmented_sentences.append(' '.join(a_words))

	#ri
	if (alpha_ri > 0):
		n_ri = max(1, int(alpha_ri*num_words))
		for _ in range(num_new_per_technique):
			a_words = random_insertion(words, n_ri)
			augmented_sentences.append(' '.join(a_words))

	#rs
	if (alpha_rs > 0):
		n_rs = max(1, int(alpha_rs*num_words))
		for _ in range(num_new_per_technique):
			a_words = random_swap(words, n_rs)
			augmented_sentences.append(' '.join(a_words))

	#rd
	if (p_rd > 0):
		for _ in range(num_new_per_technique):
			a_words = random_deletion(words, p_rd)
			augmented_sentences.append(' '.join(a_words))

	augmented_sentences = [sentence for sentence in augmented_sentences]
	shuffle(augmented_sentences)
  
	return augmented_sentences[:4]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
df=[]
df.append(Data)
num=len(Data)
for i in range(num):
  sentence=Data.iloc[i,2]
  list_of_sentence=eda(sentence)
  list_series=[]
  for sen in list_of_sentence:
    series=[Data.at[i,'Category'],Data.at[i,'CounterSpeech'],sen,Data.at[i,'date'],Data.at[i,'hasReplies'],Data.at[i,'id'],Data.at[i,'likes'],Data.at[i,'numberOfReplies'],Data.at[i,'timestamp'],Data.at[i,'user'],Data.at[i,'Community'],Data.at[i,'replies']]
    list_series.append(series)
  df.append(pd.DataFrame(list_series, columns = ['Category', 'CounterSpeech','commentText','date','hasReplies','id','likes','numberOfReplies','timestamp','user','Community','replies']))
augmented_frame=pd.concat(df)
print(augmented_frame)

   Category  CounterSpeech  ... Community replies
0   Default          False  ...      jews      []
1   Default          False  ...      lgbt      []
2       2,8           True  ...      jews      []
3         8           True  ...      jews      []
4         8           True  ...      jews      []
..      ...            ...  ...       ...     ...
3       4,8           True  ...     black      []
0       3,9           True  ...     black      []
1       3,9           True  ...     black      []
2       3,9           True  ...     black      []
3       3,9           True  ...     black      []

[69345 rows x 12 columns]


In [ ]:
augmented_frame.to_csv('augmented_data.csv', index=False)

In [ ]:
Data=pd.read_csv('augmented_data.csv')
Data

,Category,CounterSpeech,commentText,date,hasReplies,id,likes,numberOfReplies,timestamp,user,Community,replies
0,Default,False,last brave white women brain,5 years ago,0.0,Ugw-_mfnTo9SUniGGTN4AaABAg,0,0.0,2012-12-05 09:13:58.535,Ik hou van surfen,jews,[]
1,Default,False,amen pastor,2 years ago,0.0,UgiMOgO3u4akkngCoAEC,0,0.0,2015-12-05 04:47:36.420,julius smith,lgbt,[]
2,"2,8",True,kind god hates people curses kind people belie...,4 years ago,0.0,UgzemCMbg-ri1ri7sCt4AaABAg,0,0.0,2013-12-05 09:16:05.293,gary wersley,jews,[]
3,8,True,question comment racist jews smartest group pe...,3 years ago,0.0,UghFhzrURF1svHgCoAEC,0,0.0,2015-01-05 16:27:25.589,Les Ordway,jews,[]
4,8,True,israel fail nothing new israel rebellious nati...,5 years ago,0.0,Ugzr50sQdH0Ictuex214AaABAg,0,0.0,2012-12-05 09:17:14.596,HamalachMichael,jews,[]
...,...,...,...,...,...,...,...,...,...,...,...,...
69340,"4,8",True,ger think smarter people white dont people act...,4 months ago,0.0,Ugx9m9M6B-l9pX4SKG14AaABAg,0,0.0,2018-08-13 05:49:01.941,24Bryce,black,[]
69341,"3,9",True,sir would pig insult wish watch gladly pig go ...,4 months ago,0.0,UgxD1E8OwwlPqs5VOyJ4AaABAg,0,0.0,2018-08-13 05:49:01.945,RICHARD GORDON,black,[]
69342,"3,9",True,sir would pig insult wish watch gladly pig go ...,4 months ago,0.0,UgxD1E8OwwlPqs5VOyJ4AaABAg,0,0.0,2018-08-13 05:49:01.945,RICHARD GORDON,black,[]
69343,"3,9",True,sir would pig insult wish watch gladly pig go ...,4 months ago,0.0,UgxD1E8OwwlPqs5VOyJ4AaABAg,0,0.0,2018-08-13 05:49:01.945,RICHARD GORDON,black,[]


In [ ]:
Data.dropna(axis=0, inplace=True)

In [ ]:

from sklearn.model_selection import train_test_split

y_= Data.CounterSpeech
x_= Data
x_train,x_test,YTrain,YTest=train_test_split(x_,y_,test_size=0.3)

train_data = x_train.iloc[:,2].values
test_data = x_test.iloc[:,2].values


In [ ]:
x_train.to_csv('x_train_data.csv', index=False)
x_test.to_csv('x_test_data.csv',index=False)
YTrain.to_csv('y_train.csv',index=False)
YTest.to_csv('y_test.csv',index=False)


In [ ]:
print(train_data)

['wasnt even surprised said actors didnt believe minute'
 'going done ball group person evil get excuse murdering average race sadistic american sentences helping idea pass evil acts problem issue bashed life person citizen is fun people law group excuse cant get young pin race fun goliath reduce involved people issue illogical blame using'
 'gay alteration ur stupid mind forced give satan rim job hell sure isnt hate right know trying help losers like wrong i'
 ... 'shit person piece  i white kids embarrassing taking scam kids'
 'something didnt arouse filming started'
 'right doesnt would nuclear kill destroyed hypocrites needs politics great religion accident missile fantastic holy wont praise programmed nothing humanity someone land fascism sight god separate fascism fight thing everything would help deadly landed idiots']


In [ ]:
print(YTrain)

1756     False
65904     True
41009    False
30509    False
68089     True
         ...  
35036    False
16686     True
68310     True
68465    False
54373     True
Name: CounterSpeech, Length: 48540, dtype: bool


### Logistic Regresssion + Bags of Words Vector(BoWV)

In [ ]:
vect = CountVectorizer(min_df=2, ngram_range=(1, 3))
XTrain = vect.fit(train_data).transform(train_data)
XTest = vect.transform(test_data)

In [ ]:
print("Vocabulary size: {}".format(len(vect.vocabulary_)))
print("XTrain : \n{}".format(repr(XTrain)))
print("XTest : \n{}".format(repr(XTest)))
print("Features : {}".format(len(vect.get_feature_names())))

Vocabulary size: 337961
XTrain : 
<48540x337961 sparse matrix of type '<class 'numpy.int64'>'
	with 1578033 stored elements in Compressed Sparse Row format>
XTest : 
<20804x337961 sparse matrix of type '<class 'numpy.int64'>'
	with 482964 stored elements in Compressed Sparse Row format>
Features : 337961


In [ ]:
parameter_grid = {'C': [0.1, 0.2, 0.3, 0.4, 0.5], 'max_iter': [200]}
grid = GridSearchCV(LogisticRegression(), parameter_grid, cv = 5)
grid.fit(XTrain, YTrain)

GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.1, 0.2, 0.3, 0.4, 0.5], 'max_iter': [200]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
print("Best cross-validation score: {}".format(grid.best_score_))
print("Best parameters: ", grid.best_params_)
print("Best estimator: ", grid.best_estimator_)

Best cross-validation score: 0.9001854140914709
Best parameters:  {'C': 0.5, 'max_iter': 200}
Best estimator:  LogisticRegression(C=0.5, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=200,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


In [ ]:
lr = grid.best_estimator_
lr.fit(XTrain, YTrain)

LogisticRegression(C=0.5, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=200,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
ypred = lr.predict(XTest)
tn, fp, fn, tp = confusion_matrix(YTest, ypred).ravel()
print(tn, fp, fn, tp)

9631 953 1087 9133


In [ ]:
recall=tp/(tp+fn)
print(recall)

0.8936399217221135


In [ ]:
precision=tp/(tp+fp)
print(precision)

0.905512591711283


In [ ]:
accuracy=(tn+tp)/(tn+tp+fp+fn)
print(accuracy)

0.9019419342434147


In [ ]:
f1_score=(2*recall*precision)/(recall+precision)
print(f1_score)

0.8995370826356741


### Multinomial Naive Bayes + BoWV

In [ ]:

naive = MultinomialNB()
classifier_naive = naive.fit(XTrain,YTrain)
predict_naive = classifier_naive.predict(XTest)

print(predict_naive)

[False  True  True ... False False  True]


In [ ]:
tn, fp, fn, tp = confusion_matrix(YTest, predict_naive).ravel()
print(tn, fp, fn, tp)

8908 1676 913 9307


In [ ]:
recall=tp/(tp+fn)
print(recall)

0.9106653620352251


In [ ]:
precision=tp/(tp+fp)
print(precision)

0.8474005280888646


In [ ]:
accuracy=(tn+tp)/(tn+tp+fp+fn)
print(accuracy)

0.8755527783118631


In [ ]:
f1_score=(2*recall*precision)/(recall+precision)
print(f1_score)

0.8778946375512898


## Support Vector Machine + BoWV

In [ ]:
svm = LinearSVC()
classifier_SVM = svm.fit(XTrain, YTrain)
predict_SVM = svm.predict(XTest)
print(predict_SVM)
#print(classification_report(YTest, predict_SVM ))
logreg = LogisticRegression(dual=False)

[False  True  True ... False False  True]


In [ ]:
tn, fp, fn, tp = confusion_matrix(YTest, predict_SVM).ravel()
print(tn, fp, fn, tp)

9663 921 941 9279


In [ ]:
recall=tp/(tp+fn)
print(recall)

0.9079256360078278


In [ ]:
precision=tp/(tp+fp)
print(precision)

0.9097058823529411


In [ ]:
accuracy=(tn+tp)/(tn+tp+fp+fn)
print(accuracy)

0.9104979811574697


In [ ]:
f1_score=(2*recall*precision)/(recall+precision)
print(f1_score)

0.9088148873653281


## Decision Tree + Bag of Words Vector

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(XTrain, YTrain)
tree_pred = dt.predict(XTest)
print(tree_pred)

#print(classification_report(YTest,tree_pred))
#print(confusion_matrix(YTest, tree_pred))

[False  True  True ... False False  True]


In [ ]:
tn, fp, fn, tp = confusion_matrix(YTest, tree_pred).ravel()
print(tn, fp, fn, tp)

9803 781 803 9417


In [ ]:
recall=tp/(tp+fn)
print(recall)

0.9214285714285714


In [ ]:
precision=tp/(tp+fp)
print(precision)

0.9234163561482643


In [ ]:
accuracy=(tn+tp)/(tn+tp+fp+fn)
print(accuracy)

0.9238607960007691


In [ ]:
f1_score=(2*recall*precision)/(recall+precision)
print(f1_score)

0.9224213928886277


## Logistic Regression + TF-IDF

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1, 3), max_df=0.9, min_df=5, sublinear_tf=True)
XTrain = tfidf.fit(train_data).transform(train_data)
XTest = tfidf.transform(test_data)

In [ ]:
parameter_grid = {'C': [7.9,8,8.25]}
grid = GridSearchCV(LogisticRegression(max_iter = 300), parameter_grid, cv = 5)
grid.fit(XTrain, YTrain)
print("Best cross-validation score: {}".format(grid.best_score_))
print("Best parameters: ", grid.best_params_)
print("Best estimator: ", grid.best_estimator_)

Best cross-validation score: 0.8819736299958796
Best parameters:  {'C': 8.25}
Best estimator:  LogisticRegression(C=8.25, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=300,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


In [ ]:
lr = grid.best_estimator_
lr.fit(XTrain, YTrain)

LogisticRegression(C=8.25, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=300,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
lr.predict(XTest)
print("Score: {}".format(lr.score(XTest, YTest)))

Score: 0.8887233224379927


In [ ]:
ypred = lr.predict(XTest)
tn, fp, fn, tp = confusion_matrix(YTest, ypred).ravel()
print(tn, fp, fn, tp)

9413 1171 1144 9076


In [ ]:
recall=tp/(tp+fn)
print(recall)

0.8880626223091976


In [ ]:
precision=tp/(tp+fp)
print(precision)

0.8857226505318629


In [ ]:
accuracy=(tn+tp)/(tn+tp+fp+fn)
print(accuracy)

0.8887233224379927


In [ ]:
f1_score=(2*recall*precision)/(recall+precision)
print(f1_score)

0.8868910929789416


## Multinomial Naive Bayes + TF-IDF

In [ ]:
naive = MultinomialNB()
classifier_naive = naive.fit(XTrain,YTrain)
predict_naive = classifier_naive.predict(XTest)

print(predict_naive)

[False  True  True ... False False  True]


In [ ]:
tn, fp, fn, tp = confusion_matrix(YTest, predict_naive).ravel()
print(tn, fp, fn, tp)

8604 1980 1700 8520


In [ ]:
recall=tp/(tp+fn)
print(recall)

0.8336594911937377


In [ ]:
precision=tp/(tp+fp)
print(precision)

0.8114285714285714


In [ ]:
accuracy=(tn+tp)/(tn+tp+fp+fn)
print(accuracy)

0.8231109402038069


In [ ]:
f1_score=(2*recall*precision)/(recall+precision)
print(f1_score)

0.8223938223938223


## Support Vector Machine + TF-IDF

In [ ]:
svm = LinearSVC()
classifier_SVM = svm.fit(XTrain, YTrain)
predict_SVM = svm.predict(XTest)
print(predict_SVM)
#print(classification_report(YTest, predict_SVM ))
logreg = LogisticRegression(dual=False)

[False  True  True ... False False  True]


In [ ]:
tn, fp, fn, tp = confusion_matrix(YTest, predict_SVM).ravel()
print(tn, fp, fn, tp)

9497 1087 1084 9136


In [ ]:
recall=tp/(tp+fn)
print(recall)

0.8939334637964775


In [ ]:
precision=tp/(tp+fp)
print(precision)

0.8936711337180867


In [ ]:
accuracy=(tn+tp)/(tn+tp+fp+fn)
print(accuracy)

0.8956450682561046


In [ ]:
f1_score=(2*recall*precision)/(recall+precision)
print(f1_score)

0.8938022795088784


## Decision Tree + TF-IDF

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(XTrain, YTrain)
tree_pred = dt.predict(XTest)
print(tree_pred)

[False  True  True ... False False  True]


In [ ]:
tn, fp, fn, tp = confusion_matrix(YTest, tree_pred).ravel()
print(tn, fp, fn, tp)

9648 936 910 9310


In [ ]:
recall=tp/(tp+fn)
print(recall)

0.910958904109589


In [ ]:
precision=tp/(tp+fp)
print(precision)

0.9086472769861409


In [ ]:
accuracy=(tn+tp)/(tn+tp+fp+fn)
print(accuracy)

0.9112670640261488


In [ ]:
f1_score=(2*recall*precision)/(recall+precision)
print(f1_score)

0.9098016222026777
